In [57]:
import pandas as pd

### PROBLEM 1

#### fetching data from the website and creating the dataframe

In [94]:
d=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=pd.DataFrame(data=d[0])
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [95]:
print(df.shape)
df.describe()

(287, 3)


,Postcode,Borough,Neighborhood
count,287,287,287
unique,180,12,208
top,M8Y,Not assigned,Not assigned
freq,8,77,78


In [96]:

count=0
for index, row in df.iterrows():
    if row['Borough'] == 'Not assigned':
        count=count+1
print('There are {} records that needs to be deleted'.format(count))


There are 77 records that needs to be deleted


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [97]:
df=df[df.Borough != 'Not assigned']
df.shape

(210, 3)

#### if df['Neighborhood'] is not assigned then setting it to df['Borough']

In [98]:
df.loc[df['Neighborhood'] == 'Not assigned', ['Neighborhood']] =df['Borough']

In [99]:
df.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


#### Grouping neighnourhood based on postcode and Borough 

In [101]:
aggregate_df = df.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)

In [102]:
aggregate_df.head(10)

,,Neighborhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Downtown Toronto,Queen's Park
M9A,Queen's Park,Queen's Park
M1B,Scarborough,"Rouge, Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens, Parkview Hill"


##### We can see above that the Neighborhood is aggregated for Postcode M6A and M1B 

#### Printing the shape of the finalized dataframe

In [105]:
aggregate_df=aggregate_df.reset_index()
aggregate_df.shape

(103, 3)

## PROBLEM 2

#### Importing the csv which has latitute and longitute

In [107]:
df= pd.read_csv('https://cocl.us/Geospatial_data')

In [108]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Creating new dataframe which contains 5 columns ( PostCode, Borough, Neighborhood, Latitude, Longitute )

In [115]:
df_new = pd.merge(left=aggregate_df,right=df, left_on='Postcode', right_on='Postal Code')
df_new.drop(['Postal Code'], axis=1, inplace=True)
df_new.shape

(103, 5)

In [117]:
df_new.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


## PROBLEM 3

In [119]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Use geopy library to get the latitude and longitude values of Toronto

In [121]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [124]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [123]:
CLIENT_ID = 'L5WUTP50AS5Y1RBJEWCZQJLB3Y5GTUY012YGKUJNXEZWINHZ' # your Foursquare ID
CLIENT_SECRET = 'F3KVQHXW1MJIQ3U511KXOR3JM2PB3NS12IODL5DKPBLNFHDH' # your Foursquare Secret
VERSION = '20190101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L5WUTP50AS5Y1RBJEWCZQJLB3Y5GTUY012YGKUJNXEZWINHZ
CLIENT_SECRET:F3KVQHXW1MJIQ3U511KXOR3JM2PB3NS12IODL5DKPBLNFHDH


#### Analyze one of the neighborhood in Toronto

In [132]:
neighbor_name=df_new.loc[0,'Neighborhood']
neighbor_lat=df_new.loc[0,'Latitude']
neighbor_long=df_new.loc[0,'Longitude']
radius= 1000
limit=100

#### Fetch results from API for the neigborhood 'Parkwoods'

In [133]:
url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,neighbor_lat,neighbor_long,radius,limit)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e0fd490f7706a001bd58f37'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 43.762258609000014,
    'lng': -79.31721997969855},
   'sw': {'lat': 43.74425859099999, 'lng': -79.34209302030145}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8991cbf964a520814232e3',
       'name': "Allwyn's Bakery",
       'location': {'address': '81 Underhill drive',
        'lat': 43.75984035203157,
        'lng': -79.32471879917513,
        'labeledLatLngs': [{'label': 'display'

In [134]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [135]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Brookbanks Park,Park,43.751976,-79.332140
2,Tim Hortons,Café,43.760668,-79.326368
3,A&W Canada,Fast Food Restaurant,43.760643,-79.326865
4,Bruno's valu-mart,Grocery Store,43.746143,-79.324630


In [136]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

29 venues were returned by Foursquare.


#### Explore the neighborhood of Parkwoods

In [139]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [140]:
toronto_venues = getNearbyVenues(names=df_new['Neighborhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riv

In [141]:
print(toronto_venues.shape)
toronto_venues.head()

(4894, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W Canada,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


#### Let's check how many venues were returned for each neighborhood

In [142]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,49,49,49,49,49,49
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",31,31,31,31,31,31
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",17,17,17,17,17,17
"Alderwood, Long Branch",27,27,27,27,27,27
"Bathurst Manor, Downsview North, Wilson Heights",30,30,30,30,30,30
Bayview Village,12,12,12,12,12,12
"Bedford Park, Lawrence Manor East",38,38,38,38,38,38
Berczy Park,100,100,100,100,100,100


#### unique categories for the venues

In [143]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 335 uniques categories.


#### Analyzing each neighborhood

In [144]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Pl

In [145]:
to_onehot.shape

(4894, 335)

In [147]:
to_grouped=to_onehot.groupby('Neighborhood').mean().reset_index()
to_grouped

,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Pl

In [148]:
to_grouped.shape

(101, 335)

#### Top 5 venues for each neighborhood

In [150]:
num_top_venues = 5

for hood in to_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = to_grouped[to_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2      Theater  0.04
3        Hotel  0.04
4   Restaurant  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.12
1         Coffee Shop  0.04
2          Restaurant  0.04
3              Bakery  0.04
4      Sandwich Place  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0    Chinese Restaurant  0.19
1     Korean Restaurant  0.06
2          Noodle House  0.06
3                  Park  0.06
4  Fast Food Restaurant  0.06


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place  0.18
1         Grocery Store  0.18
2   Fried Chicken Joint  0.06
3              Bus Line  0.06
4  Fast Food Restaurant  0.06


----Alderwood, Long Branch----
               venue  freq
0           Pharmacy  0.11
1     Discount Stor

                      venue  freq
0               Coffee Shop  0.18
1                    Bakery  0.09
2            Sandwich Place  0.09
3                      Park  0.09
4  Mediterranean Restaurant  0.09


----East Birchmount Park, Ionview, Kennedy Park----
                  venue  freq
0    Chinese Restaurant  0.13
1           Coffee Shop  0.13
2        Discount Store  0.10
3  Fast Food Restaurant  0.06
4         Grocery Store  0.06


----East Toronto----
                  venue  freq
0                  Café  0.07
1           Coffee Shop  0.07
2      Greek Restaurant  0.06
3           Pizza Place  0.04
4  Ethiopian Restaurant  0.03


----Emery, Humberlea----
               venue  freq
0  Convenience Store  0.11
1       Intersection  0.11
2             Bakery  0.11
3        Golf Course  0.11
4     Discount Store  0.11


----Fairview, Henry Farm, Oriole----
                  venue  freq
0           Coffee Shop  0.11
1        Clothing Store  0.11
2  Fast Food Restaurant  0.07
3          

         venue  freq
0  Coffee Shop  0.08
1         Café  0.07
2        Hotel  0.05
3       Bakery  0.04
4   Restaurant  0.04


----Stn A PO Boxes 25 The Esplanade----
                 venue  freq
0          Coffee Shop  0.12
1                 Café  0.06
2  Japanese Restaurant  0.05
3           Restaurant  0.05
4             Beer Bar  0.04


----Studio District----
         venue  freq
0  Coffee Shop  0.07
1          Bar  0.05
2         Café  0.05
3       Bakery  0.04
4      Brewery  0.04


----The Annex, North Midtown, Yorkville----
                           venue  freq
0                           Café  0.05
1  Vegetarian / Vegan Restaurant  0.05
2                            Pub  0.04
3                    Coffee Shop  0.04
4             Italian Restaurant  0.04


----The Beaches----
         venue  freq
0  Coffee Shop  0.07
1  Pizza Place  0.07
2          Pub  0.07
3        Beach  0.04
4          Bar  0.04


----The Beaches West, India Bazaar----
               venue  freq
0  Indian 

In [152]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [153]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = to_grouped['Neighborhood']

for ind in np.arange(to_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Theater,Hotel,Restaurant,Cosmetics Shop,Concert Hall,Steakhouse,Gym,Movie Theater
1,Agincourt,Chinese Restaurant,Coffee Shop,Bakery,Sandwich Place,Shopping Mall,Restaurant,Caribbean Restaurant,Lounge,Shanghai Restaurant,Filipino Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Fast Food Restaurant,Pizza Place,Noodle House,Korean Restaurant,Park,Hobby Shop,Coffee Shop,Event Space,Malay Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Fried Chicken Joint,Park,Electronics Store,Sandwich Place,Beer Store,Pharmacy,Hardware Store,Caribbean Restaurant
4,"Alderwood, Long Branch",Discount Store,Pharmacy,Convenience Store,Pizza Place,Shopping Mall,Liquor Store,Garden Center,Gas Station,Skating Rink,Donut Shop


#### Clustering neighborhood

In [154]:
# set number of clusters
kclusters = 5

grouped_clustering = to_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 2, 2, 2, 2, 0, 0, 0])

In [155]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

data_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
data_merged = data_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

data_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Convenience Store,Bus Stop,Pharmacy,Shopping Mall,Tennis Court,Fish & Chips Shop,Food & Drink Shop,Café,Skating Rink
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Gym / Fitness Center,French Restaurant,Hockey Arena,Portuguese Restaurant,Sporting Goods Shop,Lounge,Furniture / Home Store,Playground,Golf Course
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Café,Pub,Diner,Italian Restaurant,Breakfast Spot,Bakery,Theater,Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,2.0,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Vietnamese Restaurant,Fried Chicken Joint,Restaurant,Dessert Shop,Mediterranean Restaurant,Bowling Alley,Boutique
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Park,Gastropub,Japanese Restaurant,Café,Burger Joint,Sushi Restaurant,Italian Restaurant,Ramen Restaurant,Shopping Mall


In [173]:
import math

In [176]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Neighborhood'], data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if cluster== 0.0 or math.isnan(cluster):
        clus=0
    else:
        clus=int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clus -1],
        fill=True,
        fill_color=rainbow[clus -1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining the clusters

In [177]:
data_merged.loc[data_merged['Cluster Labels'] == 0, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Coffee Shop,Gym / Fitness Center,French Restaurant,Hockey Arena,Portuguese Restaurant,Sporting Goods Shop,Lounge,Furniture / Home Store,Playground,Golf Course
2,Downtown Toronto,0.0,Coffee Shop,Park,Café,Pub,Diner,Italian Restaurant,Breakfast Spot,Bakery,Theater,Restaurant
4,Downtown Toronto,0.0,Coffee Shop,Park,Gastropub,Japanese Restaurant,Café,Burger Joint,Sushi Restaurant,Italian Restaurant,Ramen Restaurant,Shopping Mall
5,Queen's Park,0.0,Coffee Shop,Park,Gastropub,Japanese Restaurant,Café,Burger Joint,Sushi Restaurant,Italian Restaurant,Ramen Restaurant,Shopping Mall
7,North York,0.0,Pizza Place,Japanese Restaurant,Coffee Shop,Burger Joint,Basketball Court,Thai Restaurant,Mobile Phone Shop,Bar,Salad Place,Supermarket
9,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Italian Restaurant,Diner,Sushi Restaurant,Gastropub,Tea Room
13,North York,0.0,Coffee Shop,Restaurant,Gym,Supermarket,American Restaurant,Electronics Store,Japanese Restaurant,Beer Store,Asian Restaurant,Sushi Restaurant
14,East York,0.0,Coffee Shop,Sandwich Place,Pizza Place,Skating Rink,Thai Restaurant,Park,Liquor Store,Bus Line,Farmers Market,Bus Stop
15,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Restaurant,Bakery,Cosmetics Shop,Seafood Restaurant,Breakfast Spot,Italian Restaurant,Diner
19,East Toronto,0.0,Coffee Shop,Pizza Place,Pub,Bar,Breakfast Spot,Beach,Japanese Restaurant,Bakery,Caribbean Restaurant,Tea Room


In [178]:
data_merged.loc[data_merged['Cluster Labels'] == 1, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,1.0,Park,Pool,Farm,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


In [179]:
data_merged.loc[data_merged['Cluster Labels'] == 2, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Park,Convenience Store,Bus Stop,Pharmacy,Shopping Mall,Tennis Court,Fish & Chips Shop,Food & Drink Shop,Café,Skating Rink
3,North York,2.0,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Vietnamese Restaurant,Fried Chicken Joint,Restaurant,Dessert Shop,Mediterranean Restaurant,Bowling Alley,Boutique
6,Scarborough,2.0,Fast Food Restaurant,Bakery,Coffee Shop,Martial Arts Dojo,Fruit & Vegetable Store,Caribbean Restaurant,Chinese Restaurant,Paper / Office Supplies Store,Sandwich Place,Trail
8,East York,2.0,Brewery,Fast Food Restaurant,Pizza Place,Construction & Landscaping,Gym / Fitness Center,Soccer Stadium,Breakfast Spot,Gastropub,Bakery,Coffee Shop
10,North York,2.0,Fast Food Restaurant,Grocery Store,Furniture / Home Store,Gas Station,Park,Italian Restaurant,Coffee Shop,Photography Lab,Latin American Restaurant,Tennis Court
11,Etobicoke,2.0,Park,Pizza Place,Hotel,Fish & Chips Shop,Mexican Restaurant,Bank,Grocery Store,Gym,Clothing Store,Theater
16,York,2.0,Coffee Shop,Pizza Place,Bagel Shop,Grocery Store,Tennis Court,Korean Restaurant,Bakery,Bank,Gastropub,Optical Shop
17,Etobicoke,2.0,Coffee Shop,Pharmacy,Shopping Plaza,Fish & Chips Shop,Liquor Store,Beer Store,Grocery Store,Transportation Service,Café,Shopping Mall
18,Scarborough,2.0,Pizza Place,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Sandwich Place,Smoothie Shop,Sports Bar,Food & Drink Shop,Fried Chicken Joint,Supermarket
21,York,2.0,Mexican Restaurant,Pharmacy,Park,Bus Stop,Beer Store,Market,Food Truck,Sporting Goods Shop,Bus Line,Falafel Restaurant


In [180]:
data_merged.loc[data_merged['Cluster Labels'] == 3, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,3.0,Vietnamese Restaurant,Baseball Field,Restaurant,Yoga Studio,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


In [181]:
data_merged.loc[data_merged['Cluster Labels'] == 4, data_merged.columns[[1] + list(range(5, data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,4.0,Breakfast Spot,Playground,Burger Joint,Italian Restaurant,Park,Creperie,Farmers Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant
101,Etobicoke,4.0,Park,Italian Restaurant,Shopping Mall,Eastern European Restaurant,Ice Cream Shop,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
